## Example solutions to Intro_ANN exercises

    1. Compute the F1 (or other accuracy) scores for the TF ANN

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_pred, y_test))

    2. Explore whether the TF ANN gives overall better accuracy if PCA feature extraction is used instead of raw images

In [ ]:
from sklearn.decomposition import PCA

N = 12
alpha = 0.01
num_steps = 15000 
batch_size = 32 
display_step = 1000

A = []

for num_input in np.arange(1,200,10):
    print(num_input)
    pca  =  PCA(n_components =num_input)
    ## fit to data
    pca.fit(X_train)
    ## apply the transformation
    images_pca = pca.transform(X_train)

    images_test_pca = pca.transform(X_test)
    
    # tf Graph input
    X = tf.placeholder("float", [None, num_input])
    Y = tf.placeholder("float", [None, N])    
   
    # Store layers weight & bias# Store 
    weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, N]))
    }
    biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([N]))
    }

    # Construct model
    logits = neural_net(X)

    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y))
    optimizer = tf.train.AdamOptimizer(learning_rate=alpha)
    train_op = optimizer.minimize(loss_op)

    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()    

    # Start training
    with tf.Session() as sess:

        # Run the initializer
        sess.run(init)

        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
        for step in range(1, num_steps+1):
            ind = r.choice(np.arange(np.shape(images_pca)[0]),batch_size)
            batch_x = images_pca[ind]
            batch_y = y_v_train[ind]
            # Run optimization op (backprop)
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
            if step % display_step == 0 or step == 1:
                # Calculate batch loss and accuracy
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})

        print("Optimization Finished!")

        # Calculate accuracy for test images
        A.append(sess.run(accuracy, feed_dict={X: images_test_pca, Y: y_v_test}))

In [ ]:
plt.plot(np.arange(1,200,10), A, 'k-o')
plt.axhline(y=.36, color='r')
plt.xlabel('Number of principal components')
plt.ylabel('Average accuracy')